In [2]:

import sys
import os
from pathlib import Path
from dotenv import load_dotenv

working_dir = Path.cwd()
while working_dir.name != 'CausalMTR-BC':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'CausalMTR-BC' not found")
os.chdir(working_dir)

# Load env variables from config/.env 
dotenv_path = Path("configs/.env")
load_dotenv(dotenv_path=dotenv_path)

# GPUDRIVE_PATH = os.environ.get("GPUDRIVE_PATH")
# sys.path.append(os.path.abspath(GPUDRIVE_PATH))

from gpudrive.env.config import EnvConfig, SceneConfig
from gpudrive.env.env_torch import GPUDriveTorchEnv
from gpudrive.visualize.utils import img_from_fig
from gpudrive.env.dataset import SceneDataLoader

In [4]:
import gpudrive
print(gpudrive.__file__)

/scratch/cluster/abitpal/CausalMTR-BC/gpudrive/gpudrive/__init__.py


In [3]:
DYNAMICS_MODEL = "delta_local" # "delta_local" / "state" / "classic"
DATA_PATH = "data/processed/examples" # Your data path
MAX_NUM_OBJECTS = 6
NUM_ENVS = 2
DEVICE = "cuda:0"

# Configs
env_config = EnvConfig(dynamics_model=DYNAMICS_MODEL)

In [4]:
# Make dataloader
data_loader = SceneDataLoader(
    root="gpudrive/data/processed/examples", # Path to the dataset
    batch_size=NUM_ENVS, # Batch size, you want this to be equal to the number of worlds (envs) so that every world receives a different scene
    dataset_size=NUM_ENVS, # Total number of different scenes we want to use
    sample_with_replacement=False, 
    seed=42, 
    shuffle=True,   
)

print("Making the environment")

# Make environment
env = GPUDriveTorchEnv(
    config=env_config,
    data_loader=data_loader,
    max_cont_agents=MAX_NUM_OBJECTS, # Maximum number of agents to control per scenario
    device="cuda", 
    action_type="continuous" # "continuous" or "discrete"
)

Making the environment
Compiling GPU engine code:
/scratch/cluster/abitpal/CausalMTR-BC/gpudrive/external/madrona/src/mw/device/memory.cpp
/scratch/cluster/abitpal/CausalMTR-BC/gpudrive/external/madrona/src/mw/device/state.cpp
/scratch/cluster/abitpal/CausalMTR-BC/gpudrive/external/madrona/src/mw/device/crash.cpp
/scratch/cluster/abitpal/CausalMTR-BC/gpudrive/external/madrona/src/mw/device/consts.cpp
/scratch/cluster/abitpal/CausalMTR-BC/gpudrive/external/madrona/src/mw/device/taskgraph.cpp
/scratch/cluster/abitpal/CausalMTR-BC/gpudrive/external/madrona/src/mw/device/taskgraph_utils.cpp
/scratch/cluster/abitpal/CausalMTR-BC/gpudrive/external/madrona/src/mw/device/sort_archetype.cpp
/scratch/cluster/abitpal/CausalMTR-BC/gpudrive/external/madrona/src/mw/device/host_print.cpp
/scratch/cluster/abitpal/CausalMTR-BC/gpudrive/external/madrona/src/mw/../common/hashmap.cpp
/scratch/cluster/abitpal/CausalMTR-BC/gpudrive/external/madrona/src/mw/../common/navmesh.cpp
/scratch/cluster/abitpal/Causa

/scratch/cluster/abitpal/CausalMTR-BC/gpudrive/src/level_gen.cpp(298): warning #177-D: function "madrona_gpudrive::createFloorPlane" was declared but never referenced
  static void createFloorPlane(Engine &ctx)
              ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"





In [4]:
%load_ext autoreload
%autoreload 2

from cmtr_bc.waymo_dataset import WaymoDataset
import yaml 
from easydict import EasyDict
import logging
import torch
import numpy as np

def getconfig(cfg_file):
    with open(cfg_file, 'r') as f:
        try:
            config = yaml.load(f, Loader=yaml.FullLoader)
        except:
            config = yaml.load(f)

    return EasyDict(config)

logger = logging.getLogger(__name__)

config = getconfig("configs/config.yaml")
waymo_dataset = WaymoDataset(config.DATA_CONFIG, test_mode=1, logger=logger)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from cmtr_bc.waymo_iterator import TrajectoryIterator
from torch.utils.data import DataLoader

traj_data_loader = TrajectoryIterator(env, data_loader, cmtr=True, waymo_dataset=waymo_dataset, prior_frame=10, future_frame=10)
train_iterator = iter(DataLoader(traj_data_loader, batch_size=1, collate_fn=traj_data_loader.collate_batch))

NameError: name 'env' is not defined

In [63]:
batch_data = next(train_iterator)

INFO:root:Generating expert actions and observations for 2 worlds 

INFO:root:Using continuous expert actions... 



DATA BATCH
['gpudrive/data/processed/examples/tfrecord-00000-of-01000_4.json', 'gpudrive/data/processed/examples/tfrecord-00000-of-01000_325.json']


In [56]:
from mtr.models import model as model_utils

In [57]:
print(config.MODEL)
model = model_utils.MotionTransformer(config=config.MODEL).to("cuda:1")

{'CONTEXT_ENCODER': {'NAME': 'MTREncoder', 'NUM_OF_ATTN_NEIGHBORS': 16, 'NUM_INPUT_ATTR_AGENT': 28, 'NUM_INPUT_ATTR_MAP': 13, 'NUM_CHANNEL_IN_MLP_AGENT': 256, 'NUM_CHANNEL_IN_MLP_MAP': 64, 'NUM_LAYER_IN_MLP_AGENT': 3, 'NUM_LAYER_IN_MLP_MAP': 5, 'NUM_LAYER_IN_PRE_MLP_MAP': 3, 'D_MODEL': 128, 'NUM_ATTN_LAYERS': 6, 'NUM_ATTN_HEAD': 8, 'DROPOUT_OF_ATTN': 0.1, 'USE_LOCAL_ATTN': True}, 'MOTION_DECODER': {'NAME': 'MTRDecoder', 'OBJECT_TYPE': ['TYPE_VEHICLE', 'TYPE_PEDESTRIAN', 'TYPE_CYCLIST'], 'CENTER_OFFSET_OF_MAP': [30.0, 0], 'NUM_FUTURE_FRAMES': 10, 'NUM_MOTION_MODES': 6, 'INTENTION_POINTS_FILE': 'data/waymo/cluster_64_center_dict.pkl', 'D_MODEL': 256, 'NUM_DECODER_LAYERS': 6, 'NUM_ATTN_HEAD': 8, 'MAP_D_MODEL': 256, 'DROPOUT_OF_ATTN': 0.1, 'NUM_BASE_MAP_POLYLINES': 256, 'NUM_WAYPOINT_MAP_POLYLINES': 128, 'LOSS_WEIGHTS': {'cls': 1.0, 'reg': 1.0, 'vel': 0.5}, 'NMS_DIST_THRESH': 2.5}}


In [58]:
model.set_epoch(0)

In [59]:
output = model(batch_data[0])

In [60]:
output

(tensor(294.7106, device='cuda:1', grad_fn=<AddBackward0>),
 {'loss_layer0': 268.0614013671875,
  'loss_layer0_reg_gmm': 113.82169342041016,
  'loss_layer0_reg_vel': 20.613134384155273,
  'loss_layer0_cls': 4.175830364227295,
  'loss_layer1': 262.7039489746094,
  'loss_layer1_reg_gmm': 107.17525482177734,
  'loss_layer1_reg_vel': 20.880924224853516,
  'loss_layer1_cls': 4.207742691040039,
  'loss_layer2': 249.1282501220703,
  'loss_layer2_reg_gmm': 92.96700286865234,
  'loss_layer2_reg_vel': 20.76829719543457,
  'loss_layer2_cls': 4.231029510498047,
  'loss_layer3': 257.2741394042969,
  'loss_layer3_reg_gmm': 103.11598205566406,
  'loss_layer3_reg_vel': 20.674022674560547,
  'loss_layer3_cls': 4.171379566192627,
  'loss_layer4': 263.98602294921875,
  'loss_layer4_reg_gmm': 107.98799896240234,
  'loss_layer4_reg_vel': 20.783023834228516,
  'loss_layer4_cls': 4.225468635559082,
  'loss_layer5': 251.32913208007812,
  'loss_layer5_reg_gmm': 96.01195526123047,
  'loss_layer5_reg_vel': 20.82